# Setting up environment

In [1]:
# Installing mmseg (or Clone mmseg repo & setup) 
!rm -rf mmsegmentation
!git clone https://github.com/LARC-CMU-SMU/FoodSeg103-Benchmark-v1.git  
%cd FoodSeg103-Benchmark-v1                                                 
!pip install -e .
!pip install -r requirements/optional.txt

# Install PyTorch
!pip install -U torch==1.5.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
# Install MMCV (+ addict & yapf)
!pip install mmcv-full==1.3.0+torch1.5.0+cu101 -f https://download.openmmlab.com/mmcv/dist/index.html 
# Installing timm
!pip install timm

# Clone Volume Estimation Repo  #%cd ..
!git clone https://github.com/fatimamds/dietlane-app-volume
!mkdir input/
# Install labelme
!pip install labelme

# Install Flask & Pyngrok for API
!pip install flask-ngrok

Cloning into 'FoodSeg103-Benchmark-v1'...
remote: Enumerating objects: 775, done.
remote: Counting objects: 100% (775/775), done.
remote: Compressing objects: 100% (372/372), done.
remote: Total 775 (delta 421), reused 728 (delta 394), pack-reused 0
Receiving objects: 100% (775/775), 3.63 MiB | 13.56 MiB/s, done.
Resolving deltas: 100% (421/421), done.
/content/FoodSeg103-Benchmark-v1
Obtaining file:///content/FoodSeg103-Benchmark-v1
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=19964479ce699fab4abb54c1510aec08ac8cac7b19950dd0f2fef024ee3b8ef5
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built terminaltables
  Running setup.py develop for mmsegmentation
     |████████████████████████████████| 481kB 7.6MB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
     |████████████████████████████████| 81kB 8.2MB/s 
     |████████████████████████████████| 92kB 

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 703.8MB 25kB/s 
     |████████████████████████████████| 6.6MB 61.1MB/s 
ERROR: torchtext 0.10.0 has requirement torch==1.9.0, but you'll have torch 1.5.0+cu101 which is incompatible.
  Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Found existing installation: torchvision 0.10.0+cu102
    Uninstalling torchvision-0.10.0+cu102:
      Successfully uninstalled torchvision-0.10.0+cu102
Looking in links: https://download.openmmlab.com/mmcv/dist/index.html
     |████████████████████████████████| 24.4MB 125kB/s 
     |████████████████████████████████| 194kB 8.4MB/s 
     |████████████████████████████████| 378kB 7.5MB/s 
Cloning into 'dietlane-app-volume'...
remote: Enumerating objects: 110, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (87/87), done.
remote: 

###### Checking Environment Installation & Importing Modules

In [2]:
# Check nvcc version - NVIDIA CUDA Compiler
!nvcc -V
# Check GCC version
!gcc --version

# Check Pytorch installation
import os
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMSegmentation installation
import mmseg
from mmseg.apis import inference_segmentor, init_segmentor, show_result_pyplot
from mmseg.core.evaluation import get_palette
print(mmseg.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

1.5.0+cu101 True
0.11.0


# Fetching trained networks

In [3]:
# Download our trained SEGMENTATION model (Encoder: ReLeM-R-50 |	Decoder: CCNet)
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=18XQnJFoxLF_2wFtFmcbQktJReOm-o64X' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=18XQnJFoxLF_2wFtFmcbQktJReOm-o64X" -O iter_80000.pth && rm -rf /tmp/cookies.txt
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=16bM9qJnAfDd_mELOZKjLvnI_taZYWqaH' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=16bM9qJnAfDd_mELOZKjLvnI_taZYWqaH" -O ccnet_r50-d8_512x1024_80k.py && rm -rf /tmp/cookies.txt

# Download our trained DEPTH model (Senet)
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1QaUkdOiGpMuzMeWCGbey0sT0wXY0xtsj' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1QaUkdOiGpMuzMeWCGbey0sT0wXY0xtsj" -O pretrained_model.zip && rm -rf /tmp/cookies.txt
!unzip pretrained_model.zip  #unzipping
!rm -r pretrained_model.zip  #removing zipped file to free space

--2021-07-06 18:12:19--  https://docs.google.com/uc?export=download&confirm=uijc&id=18XQnJFoxLF_2wFtFmcbQktJReOm-o64X
Resolving docs.google.com (docs.google.com)... 74.125.135.100, 74.125.135.113, 74.125.135.102, ...
Connecting to docs.google.com (docs.google.com)|74.125.135.100|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0g-50-docs.googleusercontent.com/docs/securesc/2e0ifvrlcqml0jorn9uaujugqc62e320/fir2o4rqhk3j5lag5jc28ejnud0lh93u/1625595075000/15780393842409038990/10723609525038516285Z/18XQnJFoxLF_2wFtFmcbQktJReOm-o64X?e=download [following]
--2021-07-06 18:12:19--  https://doc-0g-50-docs.googleusercontent.com/docs/securesc/2e0ifvrlcqml0jorn9uaujugqc62e320/fir2o4rqhk3j5lag5jc28ejnud0lh93u/1625595075000/15780393842409038990/10723609525038516285Z/18XQnJFoxLF_2wFtFmcbQktJReOm-o64X?e=download
Resolving doc-0g-50-docs.googleusercontent.com (doc-0g-50-docs.googleusercontent.com)... 74.125.142.132, 2607:f8b0:400e:c08::84
Connecting

###### Building our Food Segmentation Model

In [ ]:
# build the model from a config file and a checkpoint file
config_file = './ccnet_r50-d8_512x1024_80k.py' 
checkpoint_file = './iter_80000.pth'
model = init_segmentor(config_file, checkpoint_file, device='cuda:0')

# Executing program

In [17]:
def predict_mask():
  # test a single image
  img = 'input/test.png'    #img = Image.open(requests.get('http://images.xyz.com/9.jpg', stream=True).raw)  #import requests
  result = inference_segmentor(model, img)
  # save the visualization results to image files
  model.show_result(img, result, out_file='demo/result.png')

def find_vol():
  execfile('dietlane-app-volume/masktojson.py') 
  execfile('dietlane-app-volume/demo.py')
  with open('output/volume.json') as json_file:
    data = json.load(json_file)  # returns JSON object as a dictionary
  os.remove("input/test.png")
  os.remove("input/test.json")
  return data

dietlane-app-volume/demo.py:64: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  image = torch.autograd.Variable(image, volatile=True).cuda()
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2796: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2973: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
Volume:
{'milk': 45.138711629698555, 'strawberry': 58.97066539926101, 'pork': 112.24262961009646, 'rice': 122.91126068590586, 'cabbage': 381.403468889156}
unit: cm^3


# Setting up Flask API

In [ ]:
from flask import Flask
from flask import jsonify, make_response, request
from flask_ngrok import run_with_ngrok

import numpy as np 
import cv2
import base64

#os.environ["FLASK_ENV"] = "development"

# Running the Flask app
app = Flask(__name__)
run_with_ngrok(app)

# Define Flask routes
@app.route("/", methods=['GET'])
def index():
  return "Hello from Colab!"

@app.route("/trialroute")
def postyy():
  return "Another hi"

@app.route('/uploadimage' , methods=['POST'])
def predict_vol():
  img_str = request.json['image']
  org_str = base64.b64decode(img_str)
  npimg = np.fromstring(org_str, np.uint8)
  img = cv2.imdecode(npimg,cv2.IMREAD_COLOR)
  cv2.imwrite('input/test.png', img)
  predict_mask()
  vol_data = find_vol()
  print(vol_data)
  return jsonify(vol_data)

if __name__ == '__main__':
  app.run()  #app.run(Debug=True)   #if os.environ = development

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://e2ca4da8adc4.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [07/Jul/2021 00:15:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jul/2021 00:15:20] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [07/Jul/2021 00:15:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Jul/2021 00:15:27] "GET /posty HTTP/1.1" 200 -
127.0.0.1 - - [07/Jul/2021 00:15:27] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [07/Jul/2021 00:15:29] "GET /posty HTTP/1.1" 200 -
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
[ERROR  ] app:log_exception:1892 - Exception on /uploadimage [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-pa